# This notebook contains:
* Functions to create an xarray dataset and netcdf file for a particular BARRA2 variable over JFMA

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings
import sys
import metpy
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy.calc as mpcalc
import pandas as pd
from netCDF4 import Dataset
import os
import glob
from datetime import datetime
import seaborn as sns
import netCDF4
from netCDF4 import Dataset
from metpy.units import units
import dask
import xarray as xr
from shapely import Polygon
import regionmask
import geopandas as gpd
import dask
from scipy.stats import circmean

In [50]:
# near surface:
def file_paths(variable):
    list = ["202201-202201","202202-202202","202101-202101","202102-202102","202103-202103","202104-202104","202001-202001","202002-202002",
           "202003-202003","202004-202004","201901-201901","201902-201902","201903-201903","201904-201904","201801-201801","201802-201802",
           "201803-201803","201804-201804","201701-201701","201702-201702","201703-201703","201704-201704","201601-201601","201602-201602",
           "201603-201603","201604-201604","201501-201501","201502-201502","201503-201503","201504-201504","201401-201401","201402-201402",
           "201403-201403","201404-201404","201301-201301","201302-201302","201303-201303","201304-201304","201201-201201","201202-201202",
           "201203-201203","201204-201204"]
    list.sort()
    file_paths = []
    for i in list:
        fp = "/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/1hr/"+variable+"/v20231001/"
        all_files = [fp+variable+"_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_"+i+".nc"]
        file_paths.extend(all_files)
    file_paths.sort()
    return file_paths  
u_surface = file_paths("uas") # in m/s
v_surface = file_paths("vasmean") # in m/s
u100m = file_paths("ua100m") # in m/s
v100m = file_paths("va100m") # in m/s
wind_speed = file_paths("sfcWind") # in m/s
sfc_temp = file_paths("tas") # in K
u850 = file_paths("ua850") # in m/s
v850 = file_paths("va850") #  in m/s
u500 = file_paths("ua500") #  in m/s
v500 = file_paths("va500") #  in m/s

In [51]:
%%time
def open_and_select_lat(file_path,variable):
    with xr.open_mfdataset(file_path, engine="h5netcdf", chunks="auto") as ds:
        towns = ds[variable].sel(lat=-19.4198,method="nearest").sel(lon=slice(143,152))
        cairns = ds[variable].sel(lat=-16.8736,method="nearest").sel(lon=slice(143,152))                     
        willis = ds[variable].sel(lat=-16.2874,method="nearest").sel(lon=slice(143,152)) 
        return (towns*units("m/s")).compute(),(cairns*units("m/s")).compute(),(willis*units("m/s")).compute()
# u100T,u100C,u100W = open_and_select_lat(u100m,"ua100m")
# v100T,v100C,v100W = open_and_select_lat(v100m,"va100m")
uuT,uuC,uuW = open_and_select_lat(u850,"ua850")
vvT,vvC,vvW = open_and_select_lat(va850,"va850")
# tas_meanT,tas_meanC,tas_meanW = open_and_select_lat(sfc_temp,"tas")

NameError: name 'va850' is not defined

In [3]:
%%time
# function for calculating mean 850hPa wind regimes over the radar domains
def open_and_calc_mean(file_path,variable):
    with xr.open_mfdataset(file_path, engine="h5netcdf", chunks="auto") as ds:
        towns = ds[variable].sel(lat=slice(-20.768799,-18.0708),lon=slice(145.12054,147.9812)).mean(dim=["lat","lon"])
        cairns = ds[variable].sel(lat=slice(-18.165955,-15.468018),lon=slice(144.27374,147.09222)).mean(dim=["lat","lon"])                  
        willis = ds[variable].sel(lat=slice(-17.636353,-14.938416),lon=slice(148.55927,151.36993)).mean(dim=["lat","lon"])
        return (towns*units("m/s")).compute(),(cairns*units("m/s")).compute(),(willis*units("m/s")).compute()
uuT,uuC,uuW = open_and_calc_mean(u500,"ua500")
vvT,vvC,vvW = open_and_calc_mean(v500,"va500")

CPU times: user 7min 2s, sys: 2min 11s, total: 9min 14s
Wall time: 14min 19s


In [4]:
%%time
# wind speeds and calculations
towns_wind_dir = metpy.calc.wind_direction(uuT,vvT,convention='from')
towns_wind_speed = metpy.calc.wind_speed(uuT,vvT)
cairns_wind_dir = metpy.calc.wind_direction(uuC,vvC,convention='from')
cairns_wind_speed = metpy.calc.wind_speed(uuC,vvC)
willis_wind_dir = metpy.calc.wind_direction(uuW,vvW,convention='from')
willis_wind_speed = metpy.calc.wind_speed(uuW,vvW)

CPU times: user 16.4 ms, sys: 850 µs, total: 17.2 ms
Wall time: 15.5 ms


In [41]:
# change from pint quantity to array for open_and_calc_mean arrays
time_values = towns_wind_dir.time.values
towns_wind_speed_array = xr.DataArray(towns_wind_speed.values,dims=('time'), coords={'time': time_values})
towns_wind_dir_array = xr.DataArray(towns_wind_dir.values,dims=('time'), coords={'time': time_values})
cairns_wind_speed_array = xr.DataArray(cairns_wind_speed.values,dims=('time'), coords={'time': time_values})
cairns_wind_dir_array = xr.DataArray(cairns_wind_dir.values,dims=('time'), coords={'time': time_values})
willis_wind_speed_array = xr.DataArray(willis_wind_speed.values,dims=('time'), coords={'time': time_values})
willis_wind_dir_array = xr.DataArray(willis_wind_dir.values,dims=('time'), coords={'time': time_values})

In [42]:
# combine xarrays into xarray dataset
towns_wind = xr.Dataset({'wind_dir':towns_wind_dir_array,
                         'wind_speed':towns_wind_speed_array}, 
            attrs={'note':'JFMA mean 500hPa winds for townsville radar domain created with xarray'}) #JFMA 100m winds for townsville latitude slice (longitude 143-152deg) created with xarray
cairns_wind = xr.Dataset({'wind_dir':cairns_wind_dir_array,
                         'wind_speed':cairns_wind_speed_array}, 
            attrs={'note':'JFMA mean 500hPa winds for cairns radar domain created with xarray'})
willis_wind = xr.Dataset({'wind_dir':willis_wind_dir_array,
                         'wind_speed':willis_wind_speed_array}, 
            attrs={'note':'JFMA mean 500hPa winds for willis island radar domain created with xarray'})


In [46]:
# save surface wind speed and wind direction
willis_wind.to_netcdf('barra-2_500hPa_winds_willis.nc', format='NETCDF4', 
             encoding={'wind_dir':{
                       'shuffle':True,         # increases effectiveness of compression when True
                       'chunksizes':[1], # length of time, lat, lon
                       'zlib':True,            # compression type, required to be True to specify compression levels below
                       'complevel':5           # specifies compression levels, range(0,9) with 0: no compression, 9: fully compressed
            },
                      'wind_speed':{
                      'shuffle':True,         # increases effectiveness of compression when True
                       'chunksizes':[1], # length of time, lat, lon (82)
                       'zlib':True,            # compression type, required to be True to specify compression levels below
                       'complevel':5  
                      }})